## Тренировочный тест: Критерии Стьюдента
### Вопрос 1.
Пусть $t$ — значение статистики критерия Стьюдента для независимых выборок, $nu$ — соответствующее число степеней свободы. Какая из строчек кода вычисляет достигаемый уровень значимости $t$-критерия при альтернативе $H_1:\mu_1>\mu_2$?
```
1-stats.t.cdf(t, nu)
```
### Вопрос 2.
Выберите ситуации, в которых допустимо проверять нулевую гипотезу $H_0:\mu_1=\mu_2$ против односторонней альтернативы $H_1:\mu_1>\mu_2$
- Признаки устроены так, что $\mu_1<\mu_2$ в принципе невозможно

### Вопрос 3.
Выберите верные утверждения о проблеме Беренца-Фишера.
- Приближённое решение проблемы Беренца-Фишера достаточно точно при $n_1=n_2$
- Проблема заключается в том, что не существует точного способа сравнить средние двух выборок с неизвестными дисперсиями

### Вопрос 4.
Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *

In [2]:
from numpy import sqrt
from scipy.stats import norm
x = (9.57-9.5)/(0.4/sqrt(160))
y =2*(1-norm.cdf(abs(x)))
print 'answer 4:', y

answer 4: 0.0268566955075


### Вопрос 5.
Как вы считаете, это отличие в среднем уровне кальция в крови практически значимо?
- Скорее всего нет: стандартное отклонение уровня в популяции намного больше полученного различия между средними

### Вопрос 6.
Имеются данные о стоимости и размерах 53940 бриллиантов.<br>
Отделите 25% случайных наблюдений в тестовую выборку с помощью функции `sklearn.cross_validation.train_test_split` (зафиксируйте `random state = 1`). На обучающей выборке настройте две регрессионные модели:

1. линейную регрессию с помощью `LinearRegression` без параметров
2. случайный лес с помощью RandomForestRegressor с `random_state=1`.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости $\alpha=0.05$?
- Отвергается, $p<\alpha$

In [3]:
import pandas as pd

data = pd.read_csv('diamonds.txt', sep='\t')
data.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

data_y = data['price']
data_x = data.drop(['price'], axis=1)

In [5]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split (data_x, data_y, random_state = 1, test_size=0.25)

C:\Users\Insassin\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
lm = LinearRegression().fit(x_train, y_train)
rf = RandomForestRegressor(random_state=1).fit(x_train, y_train)

In [7]:
stats.ttest_rel (abs(y_test - lm.predict(x_test)), abs(y_test - rf.predict(x_test)))

Ttest_relResult(statistic=12.745056775182125, pvalue=5.428654818173765e-37)

### Вопрос 7.
В предыдущей задаче посчитайте 95% доверительный интервал для разности средних абсолютных ошибок предсказаний регрессии и случайного леса. Чему равна его ближайшая к нулю граница? Округлите до **десятков** (поскольку случайный лес может давать немного разные предсказания в зависимости от версий библиотек, мы просим вас так сильно округлить, чтобы полученное значение наверняка совпало с нашим).

In [8]:
def abses(pred, real):
    errs = []
    for each, every in zip(pred, real):
        errs.append(abs(each - every))
    return errs

er1 = np.array(abses(lm.predict(x_test),y_test))
er2 = np.array(abses(rf.predict(x_test),y_test))
print er1.mean(), er2.mean()

890.376400429 804.702882475


In [9]:
print "95%% confidence interval: [%f, %f]" % DescrStatsW(er1 - er2).tconfint_mean()

95% confidence interval: [72.497266, 98.849770]


In [10]:
print 'answer 7:', 70

answer 7: 70
